In [ ]:
import pandas as pd
import re
from typing import Dict, Union, List

In [ ]:
colunas = ['cd_causa', 'cd_atendimento', 'ds_Acao_Judicial', 'ds_fatos', 'ds_Pedidos', 'ds_Qualificacao']

# LINHA DE SELECAO DO INPUT
df = pd.read_csv(
    "dataset_clinica20252.csv",
    sep="|",               
    encoding="utf-8"     
)

print(df.shape)
print(df.head(20))

In [ ]:
def extrair_cnpj(texto):
	rx_cnpj = r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})'
	cnpj = texto.str.extract(rx_cnpj)
	cnpj_limpo = cnpj.iloc[:,0].str.replace(r'\D', '', regex=True)
	return cnpj_limpo


In [ ]:
def extrair_valor_causa(texto: str) -> Dict[str, Union[str, float, List[str], bool, None]]:
    """
    Extrai o valor da causa de textos de petições judiciais brasileiras.

    Utiliza múltiplos padrões regex em ordem de especificidade para identificar
    o valor da causa, distinguindo-o de outros valores monetários (indenizações,
    danos morais, honorários, etc.).

    Args:
        texto (str): Texto da petição judicial (coluna ds_Pedidos ou similar).

    Returns:
        dict: Dicionário com informações da extração:
            - valor_encontrado (str): Ex: "R$ 10.000,00"
            - valor_numerico (float): Ex: 10000.00
            - padrao_usado (str)
            - confianca (str): "alta", "média" ou "baixa"
            - todos_valores (List[str])
            - sucesso (bool)
            - qtd_valores (int)
    """
    if not texto or not isinstance(texto, str) or texto.strip() == '':
        return {
            'valor_encontrado': None,
            'valor_numerico': None,
            'padrao_usado': None,
            'confianca': None,
            'todos_valores': [],
            'sucesso': False,
            'qtd_valores': 0
        }

    # Padrões em ordem de prioridade
    padroes = [
        {
            'nome': 'dar-se-á à causa',
            'regex': r'(?:dar-se-á|dar-se-a|dá-se|da-se|dão)\s+(?:à|a)\s+causa\s+o\s+valor\s+de\s+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        {
            'nome': 'valor da causa explícito',
            'regex': r'valor\s+da\s+causa[:\s]+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        {
            'nome': 'atribui-se à causa',
            'regex': r'atribui[r-]?se\s+(?:à|a)\s+causa\s+o\s+valor\s+de\s+R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        },
        {
            'nome': 'contexto causa-valor',
            'regex': r'(?:atribui|atribuir|atribuído|atribuindo).*?causa.*?valor.*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'média'
        },
        {
            'nome': 'fins fiscais',
            'regex': r'(?:para\s+fins|para\s+efeitos).*?(?:fiscais|meramente\s+fiscais).*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'média'
        },
        {
            'nome': 'seção valor da causa',
            'regex': r'(?:DO|DA)\s+VALOR\s+DA\s+CAUSA.*?R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})',
            'confianca': 'alta'
        }
    ]

    resultado = {
        'valor_encontrado': None,
        'valor_numerico': None,
        'padrao_usado': None,
        'confianca': None,
        'todos_valores': [],
        'sucesso': False,
        'qtd_valores': 0
    }

    # Captura todos os valores no formato R$ X.XXX,XX
    padrao_geral = r'R\$\s*([0-9]{1,3}(?:\.[0-9]{3})*,[0-9]{2})'
    todos_valores = re.findall(padrao_geral, texto)
    resultado['todos_valores'] = todos_valores
    resultado['qtd_valores'] = len(todos_valores)

    if not todos_valores:
        return resultado

    # Tentar encontrar por padrão específico
    for padrao in padroes:
        match = re.search(padrao['regex'], texto, re.IGNORECASE | re.DOTALL | re.MULTILINE)
        if match:
            valor_str = match.group(1)
            resultado['valor_encontrado'] = f"R$ {valor_str}"
            resultado['padrao_usado'] = padrao['nome']
            resultado['confianca'] = padrao['confianca']
            resultado['sucesso'] = True
            try:
                resultado['valor_numerico'] = float(valor_str.replace('.', '').replace(',', '.'))
            except ValueError:
                resultado['valor_numerico'] = None
            return resultado

    # Heurística: último valor encontrado (baixa confiança)
    valor_str = todos_valores[-1]
    resultado['valor_encontrado'] = f"R$ {valor_str}"
    resultado['padrao_usado'] = 'último valor (heurística)'
    resultado['confianca'] = 'baixa'
    resultado['sucesso'] = True
    try:
        resultado['valor_numerico'] = float(valor_str.replace('.', '').replace(',', '.'))
    except ValueError:
        resultado['valor_numerico'] = None

    return resultado


In [ ]:
for linha in df['ds_Pedidos'].head(50):
    for chave, valor in extrair_valor_causa(linha).items():
        df[chave] = valor
        print(f"{chave}: {valor}")
    print("-----")

for linha in df['ds_Qualificacao'].head(20):
    cnpj_extraido = extrair_cnpj(pd.Series([linha]))
    df['cnpj_extraido'] = cnpj_extraido
    print(f"CNPJ extraído: {cnpj_extraido.iloc[0]}")
    print("-----")